In [1]:
import pandas as pd
import psycopg2
import sqldf
import csv

In [2]:
host = '******.########.******.rds.amazonaws.com'
database = '******'
user = '******'
password = '******'
port = '0000'

In [3]:
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

In [4]:
cur = conn.cursor()

In [5]:
# drop_table = '''DROP TABLE IF EXISTS uber_data_dump;'''
create_table_query = '''
    CREATE TABLE IF NOT EXISTS uber_data_dump (
        VendorID integer,
        tpep_pickup_datetime timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count	integer,
        trip_distance decimal(8,4),
        pickup_longitude decimal(30,20),
        pickup_latitude	decimal(30,20),
        RatecodeID	integer,
        store_and_fwd_flag varchar(1),
        dropoff_longitude decimal(30,20),
        dropoff_latitude decimal(30,20),
        payment_type integer,
        fare_amount decimal(8,4),
        extra numeric(8,4),
        mta_tax	decimal(8,4),
        tip_amount decimal(8,4),
        tolls_amount decimal(8,4),
        improvement_surcharge decimal(8,4),
        total_amount decimal(8,4)
    );
'''
# cur.execute(drop_table)
# cur.execute(create_table_query)
# conn.commit()

In [6]:
# In Terminal:
# psql -h postgres-1.cvp1daggwpce.us-east-1.rds.amazonaws.com -U postgres -d postgres -p 5432

# \copy uber_data_dump FROM '/Users/aditi/Downloads/uber_data.csv' DELIMITER ',' CSV HEADER

In [7]:
# drop_table1 = '''DROP TABLE IF EXISTS uber_fact;'''
# drop_table2 = '''DROP TABLE IF EXISTS vendor_dim;'''
# drop_table3 = '''DROP TABLE IF EXISTS date_dim;'''
# drop_table4 = '''DROP TABLE IF EXISTS loc_dim;'''
# drop_table5 = '''DROP TABLE IF EXISTS rate_dim;'''
# drop_table6 = '''DROP TABLE IF EXISTS payment_type;'''
# cur.execute(drop_table1)
# cur.execute(drop_table2)
# cur.execute(drop_table3) 
# cur.execute(drop_table4)
# cur.execute(drop_table5)
# cur.execute(drop_table6)
conn.commit()

In [8]:
create_fact_table = '''
    CREATE TABLE IF NOT EXISTS uber_fact (
        trip_id integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        VendorID integer,
        pickup_date_id integer,
        drop_date_id integer,
        pickup_loc_id integer,
        drop_loc_id integer,
        rate_code_id integer,
        payment_type_id integer,
        passenger_count	integer,
        store_and_fwd_flag varchar(1),
        trip_distance decimal(8,4),
        fare_amount decimal(8,4),
        extra numeric(8,4),
        mta_tax	decimal(8,4),
        tip_amount decimal(8,4),
        tolls_amount decimal(8,4),
        improvement_surcharge decimal(8,4),
        total_amount decimal(8,4)
    );
'''
cur.execute(create_fact_table)
conn.commit()

In [9]:
create_dim_table = '''
    CREATE TABLE IF NOT EXISTS vendor_dim (
        VendorID integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        Vendor_name varchar(20),
        vendor_info varchar(50)
    );
'''
cur.execute(create_dim_table)
conn.commit()

In [10]:
create_dim_table = '''
    CREATE TABLE IF NOT EXISTS date_dim (
        date_id integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        date_time timestamp,
        day integer,
        month integer,
        year integer,
        hour integer,
        min integer,
        sec integer
    );
'''
cur.execute(create_dim_table)
conn.commit()

In [11]:
create_dim_table = '''
    CREATE TABLE IF NOT EXISTS loc_dim (
        loc_id integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        longitude decimal(30,20),
        latitude decimal(30,20)
    );
'''
cur.execute(create_dim_table)
conn.commit()

In [12]:
create_dim_table = '''
    CREATE TABLE IF NOT EXISTS rate_dim (
        rate_code_id integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        rate_code_desc varchar(20)
    );
'''
cur.execute(create_dim_table)
conn.commit()

In [13]:
create_dim_table = '''
    CREATE TABLE IF NOT EXISTS payment_type (
        payment_type_code integer PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
        Payment_type varchar(20)
    );
'''
cur.execute(create_dim_table)
conn.commit()

In [14]:
df = pd.read_csv("/Users/aditi/Downloads/uber_data.csv")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RatecodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

In [16]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [17]:
# reset_index(): resets the index numbers and stores the old indices in an index column
# use drop = True to avoid adding extra column to store old indices.

In [18]:
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
df['trip_id'] = df.index

In [20]:
df.tail(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
99998,2,2016-03-01 06:17:10,2016-03-01 06:22:09,1,0.92,-73.984901,40.763111,1,N,-73.970695,40.759148,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,99998
99999,1,2016-03-01 06:17:11,2016-03-01 06:22:00,1,1.00,-73.990685,40.750473,1,N,-73.980354,40.754910,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,99999


In [21]:
date_dim = pd.Series.to_frame(df['tpep_pickup_datetime'].reset_index(drop=True), name = 'tpep_datetime')

In [22]:
date_dim = pd.Series.to_frame(pd.concat([date_dim['tpep_datetime'],df['tpep_dropoff_datetime'].\
                                         reset_index(drop=True)]), name = 'tpep_datetime')


In [23]:
date_dim = date_dim.drop_duplicates().reset_index(drop = True)

In [24]:
date_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47007 entries, 0 to 47006
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   tpep_datetime  47007 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 367.4 KB


In [25]:
date_dim['date'] = date_dim['tpep_datetime'].dt.day
date_dim['month'] = date_dim['tpep_datetime'].dt.month
date_dim['year'] = date_dim['tpep_datetime'].dt.year
date_dim['hour'] = date_dim['tpep_datetime'].dt.hour
date_dim['minute'] = date_dim['tpep_datetime'].dt.minute
date_dim['second'] = date_dim['tpep_datetime'].dt.second
date_dim['date_id'] = date_dim.index

In [26]:
date_dim = date_dim[['date_id', 'tpep_datetime', 'date', 'month', 'year', 'hour', 'minute', 'second']]

In [27]:
vendor_dim = pd.Series.to_frame(df['VendorID'].drop_duplicates().reset_index(drop=True), name = 'VendorID')

In [28]:
vendor_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   VendorID  2 non-null      int64
dtypes: int64(1)
memory usage: 148.0 bytes


In [29]:
loc_dim = pd.Series.to_frame(df['pickup_longitude'].reset_index(drop=True), name = 'longitude')

In [30]:
loc_dim = pd.Series.to_frame(pd.concat([loc_dim['longitude'],df['dropoff_longitude'].reset_index(drop=True)]),name = 'longitude')


In [31]:
loc_dim_lat = pd.Series.to_frame(df['pickup_latitude'].reset_index(drop=True), name = 'latitude')

In [32]:
loc_dim_lat = pd.Series.to_frame(pd.concat([loc_dim_lat['latitude'],df['dropoff_latitude'].reset_index(drop=True)]),name = 'latitude')


In [33]:
loc_dim['latitude'] = loc_dim_lat

In [34]:
loc_dim['loc_id'] = loc_dim.index

In [35]:
loc_dim = loc_dim[['loc_id', 'longitude', 'latitude']]

In [36]:
rate_dim = pd.Series.to_frame(df['RatecodeID'].drop_duplicates().reset_index(drop=True), name = 'RatecodeID')

In [37]:
rate_dim['rate_desc'] = ['Standard rate', 'JFK', 'Newark', 'Nassau or Westchester', 'Negotiated fare', 'Group ride']
rate_dim

,RatecodeID,rate_desc
0,1,Standard rate
1,3,JFK
2,2,Newark
3,5,Nassau or Westchester
4,4,Negotiated fare
5,6,Group ride


In [38]:
payment_type_dim = pd.DataFrame([1,2,3,4,5,6], columns = ['payment_type_id'])
payment_type_dim['payment_type'] = ['Credit card', 'Cash', 'No charge', 'Dispute', 'Unknown', 'Voided trip']

In [39]:
payment_type_dim

,payment_type_id,payment_type
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [40]:
# the tables are: date_dim, vendor_dim, loc_dim, rate_dim, payment_type_dim